In [14]:
import numpy as np

import os, sys
from netCDF4 import Dataset

# MESH VARIABLES
xgrid = 99
ygrid = 99
zgrid = 1

In [64]:
def checkerboard_profile():
    xrange, yrange = xgrid, ygrid
    epsilon = 0.001
    Ax, Ay = 1, 1
    fx, fy = 1, 1
    print(f'\nUsing fx={fx} and fy={fy}\n')
    k = fx*2*np.pi/xrange #wavenumber 2pi / L
    m = fy*2*np.pi/yrange
    x=np.arange(xrange)
    y=np.arange(yrange)
    X,Y=np.meshgrid(x,y)
    phi=Ax*np.sin(k*X+epsilon)*Ay*np.sin(m*Y+epsilon)

    phi_star = phi.copy()

    max_conc = 100 # ppmv (o3) 
    min_conc = 0.03 #ppmv (o3)

    phi_star[phi_star > 0] = max_conc
    phi_star[phi_star <= 0] = min_conc

    mesh = np.zeros((xgrid, ygrid, zgrid))
    mesh[:, :, 0] = phi_star

    # transpose so z, y, x
    mesh = mesh.T

    mesh = np.ma.array(np.array([mesh]), mask=False, dtype='float32')

    return mesh

In [77]:
def create_modified_netcdf():
    # Code via 
    # https://stackoverflow.com/questions/15141563/python-netcdf-making-a-copy-of-all-variables-and-attributes-but-one
    # User Rich Signell
    
    emissions_profile = checkerboard_profile()

    toexclude = ['E_SO2']

    src = Dataset("wrfchemi_00z_d01")
    dst = Dataset("wrfchemi_00z_d01_modified", "w")

    # copy global attributes all at once via dictionary
    dst.setncatts(src.__dict__)
    # copy dimensions
    for name, dimension in src.dimensions.items():
        dst.createDimension(
            name, (len(dimension) if not dimension.isunlimited() else None))
    # copy all file data except for the excluded
    for name, variable in src.variables.items():
        if name not in toexclude:
            x = dst.createVariable(name, variable.datatype, variable.dimensions)
            dst[name][:] = src[name][:]
            # copy variable attributes all at once via dictionary
            dst[name].setncatts(src[name].__dict__)
        else:
            x = dst.createVariable(name, variable.datatype, variable.dimensions)

            # assign the profile for all timesteps (assuming time is first dimension)
            for i in range(dst[name].shape[0]):
                dst[name][i] = emissions_profile 
            # copy variable attributes all at once via dictionary
            dst[name].setncatts(src[name].__dict__)
            
    src.close()
    dst.close()
    return


In [79]:
create_modified_netcdf()


Using fx=1 and fy=1



In [71]:
src = Dataset("wrfchemi_00z_d01")

In [3]:
mod_src = Dataset("wrfchemi_00z_d01_modified")

NameError: name 'Dataset' is not defined

In [70]:
mod_src['E_SO2']

<class 'netCDF4._netCDF4.Variable'>
float32 E_SO2(Time, emissions_zdim_stag, south_north, west_east)
    MemoryOrder: XYZ
    description: EMISSIONS
    units: mol km^-2 hr^-1
    stagger: Z
    FieldType: 104
unlimited dimensions: Time
current shape = (12, 1, 99, 99)
filling on, default _FillValue of 9.969209968386869e+36 used

In [81]:
mod_src['E_SO2'][0]

masked_array(
  data=[[[1.e+02, 1.e+02, 1.e+02, ..., 3.e-02, 3.e-02, 3.e-02],
         [1.e+02, 1.e+02, 1.e+02, ..., 3.e-02, 3.e-02, 3.e-02],
         [1.e+02, 1.e+02, 1.e+02, ..., 3.e-02, 3.e-02, 3.e-02],
         ...,
         [3.e-02, 3.e-02, 3.e-02, ..., 1.e+02, 1.e+02, 1.e+02],
         [3.e-02, 3.e-02, 3.e-02, ..., 1.e+02, 1.e+02, 1.e+02],
         [3.e-02, 3.e-02, 3.e-02, ..., 1.e+02, 1.e+02, 1.e+02]]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)

In [2]:
mod_src['E_SO2'][-1]

NameError: name 'mod_src' is not defined